<a href="https://www.kaggle.com/code/wesleypdasilva/desafio-titanic-linear-regression?scriptVersionId=143573530" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# $\large{\color{CadetBlue}{\textbf{Desafio do Titanic com Regressão Linear 🚢}}}$
### De 2224 pessoas, morreram 1502;
### No dado 'treino' contém 891 registros;

Este modelo de Linear Regression não é o adequado para este tipo de dataset, ainda assim, foi feito este exercício com base em Linear Regression apenas para provar este ponto e experimentar o comportamento deste modelo.

Para este teste não foi usado os dados de testes do Kaggle, tendo em vista que eu não vou submeter este arquivo na competição do Kaggle, em vez disso, separei 30% dos dados de treino para fazer os testes.

## $\large{\color{RoyalBlue}{1.}}$ $\large{\color{CadetBlue}{\textbf{Carregando as Bibliotecas 📚}}}$

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

## $\large{\color{RoyalBlue}{2.}}$ $\large{\color{CadetBlue}{\textbf{Lendo os dados 👀}}}$

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dados = pd.read_csv("/kaggle/input/titanic/train.csv")
dados.head()

## $\large{\color{RoyalBlue}{3.}}$ $\large{\color{CadetBlue}{\textbf{Análise Exploratória dos Dados 🔎}}}$

##### $\large{\color{RoyalBlue}{3.1.}}$ $\large{\color{CadetBlue}{\textbf{Análise Descritiva dos Gêneros ⚥}}}$

###### $\large{\color{RoyalBlue}{3.1.1.}}$ $\large{\color{CadetBlue}{\textbf{Mulher 👩}}}$

In [ ]:
mulher = dados.loc[dados.Sex == "female"]["Survived"]
taxa_mulher = sum(mulher)/len(mulher)

print(len(mulher), 'totais')
print(sum(mulher), 'sobreviventes')
print("% de mulheres sobreviventes:", taxa_mulher)

###### $\large{\color{RoyalBlue}{3.1.2.}}$ $\large{\color{CadetBlue}{\textbf{Homem 👦}}}$

In [ ]:
homem = dados.loc[dados.Sex == "male"]["Survived"]
taxa_homem = sum(homem)/len(homem)

print(len(homem), 'totais')
print(sum(homem), 'sobreviventes')
print("% de homens sobreviventes:", taxa_homem)

###### $\large{\color{RoyalBlue}{3.1.3.}}$ $\large{\color{CadetBlue}{\textbf{Visualização em um Gráfico de Barras 📊}}}$

In [ ]:
plt.bar(["Homem", "Mulher"], [taxa_homem*100, taxa_mulher*100],capsize=10,color='#8400ff')

# Define o fundo como branco
plt.gca().set_facecolor('white')

# Adicionando rótulos e título ao gráfico
plt.xlabel("Gênero", color='black')
plt.ylabel("Taxa de Sobrevivência", color='black')
plt.title("Taxa de Sobrevivência por Gênero", color='black')

# Adicionando os valores das taxas acima das barras
for i, v in enumerate([taxa_homem*100, taxa_mulher*100]):
    plt.text(i, v + 0.01, f"{v:.2f}", ha="center", fontweight="bold", color='black')

# Exibindo o gráfico
plt.show()

A diferença da taxa de sobrevivência entre homens e mulheres é bem significante, mas podemos olhar outras variáveis:

##### $\large{\color{RoyalBlue}{3.2.}}$ $\large{\color{CadetBlue}{\textbf{Histogramas de Distribuição dos Sobreviventes 📊}}}$

###### $\large{\color{RoyalBlue}{3.2.1.}}$ $\large{\color{CadetBlue}{\textbf{Distribuição de Idade por Sobreviventes 👶👴}}}$

In [ ]:
# Define a paleta de cores roxas
paleta = "Set1"

plt.figure(figsize=(8, 6))
sns.histplot(data=dados, x=dados['Age'], hue=dados['Survived'], bins=20, kde=True,palette=paleta)
plt.title('Distribuição de Idade por Sobreviventes')
plt.xlabel('Idade')
plt.ylabel('Contagem')
plt.show()

Muitas crianças sobreviveram, o que faz este gráfico ter o seu início com uma subida, seguida de uma descida para então novamente subir, onde ele mantem um perfil gaussiano até por volta da idade de 50 anos, quando assim como as crianças, os idosos também passam a ser priorizados.

###### $\large{\color{RoyalBlue}{3.2.2.}}$ $\large{\color{CadetBlue}{\textbf{Distribuição de Tarifa por Sobreviventes 💲}}}$

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(data=dados, x=dados['Fare'], hue=dados['Survived'], bins=20, kde=True,palette=paleta)
plt.title('Distribuição de Tarifa por Sobreviventes')
plt.xlabel('Tarifa')
plt.ylabel('Contagem')
plt.show()

É possível perceber que a medida que a tarifa aumenta, ou seja, o preço da passagem, o número de sobreviventes aumenta proporcionalmente ao de não sobreviventes, chegando ao ponto de inverter a distribuição.

###### $\large{\color{RoyalBlue}{3.2.3.}}$ $\large{\color{CadetBlue}{\textbf{Classe de Passageiros em relação a Sobreviventes 💰}}}$

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=dados, x=dados['Pclass'], hue=dados['Survived'],palette=paleta)
plt.title('Classe de Passageiros em relação a Sobreviventes')
plt.xlabel('Classe de Passageiros')
plt.ylabel('Contagem')
plt.show()

Com um estilo de gráfico de barras, podemos ver que a terceira classe é onde tivemos mais mortes, na segunda classe o número de mortes e sobreviventes se aproxima e já na primeira classe, o número de sobreviventes é consideravelmente maior que o número de mortos.

## $\large{\color{RoyalBlue}{4.}}$ $\large{\color{CadetBlue}{\textbf{Pré-processamento dos Dados 💻}}}$

### Diferente do Random Forest, na Linear Regression é importe ter dados mais tratados para o modelo conseguir fazer os cálculos, para isso o primeiro passo que tomei foi eliminar as colunas que não fazem parte das variáveis independentes do modelo.

##### $\large{\color{RoyalBlue}{4.1.}}$ $\large{\color{CadetBlue}{\textbf{Excluindo Colunas Desnecessárias 🧹}}}$

In [ ]:
dados.drop('Name',axis = 1, inplace = True)
dados.drop('Ticket',axis = 1, inplace = True)
dados.drop('Cabin',axis = 1, inplace = True)
dados.head()

##### $\large{\color{RoyalBlue}{4.1.}}$ $\large{\color{CadetBlue}{\textbf{Verificando se há Valores Nulos ❌}}}$

In [ ]:
faltantes = (dados.isnull().sum()/len(dados['PassengerId']))*100
print(faltantes)

Prossegui em apagar todas as linhas que continham NaN.

De 891 linhas o resultado do processo diminuiu para 712 linhas.

In [ ]:
dados = dados.dropna()

## $\large{\color{RoyalBlue}{5.}}$ $\large{\color{CadetBlue}{\textbf{Variáveis categóricas em Regressão Linear 🎓}}}$

A divisão entre os gêneros é importe para o resultado do modelo, conforme mostrado na parte de Análise Descritiva dos Gêneros, mas não consigo usá-la tratando ela como uma string,já que o modelo de regressão Linear funciona apenas com variáveis contínuas, então eu resolvi tomar duas estratégias para lidar com esse problema, a primeira estratégia, para o meu tipo de conjunto de dados é a correta, tratando os valores categóricos como uma **Codificação Dummy** *(One-Hot Encoding).* 

> Nessa abordagem, você converte cada categoria da variável categórica em uma nova coluna binária (0 ou 1). Cada coluna representará uma categoria específica e indicará se essa categoria está presente (1) ou não (0) para cada observação. Isso permite que a informação categórica seja representada numericamente e seja utilizada no modelo.



&nbsp;
***

Já a segunda estratégia, incorreta, arbitrariamente eu inseri valores para cada variável, isso burla a filtro de dados contínuos mas as variáveis recebem valores que serão erroneamente correlacionados.

***
&nbsp;

Outros exemplos de estratégias disponíveis, mas que não são adequadas para estas variáveis são:

1. > Codificação Ordinal: Se a variável categórica tiver uma ordem natural (por exemplo, baixo, médio, alto), você pode atribuir valores numéricos para representar essa ordem. No entanto, isso deve ser feito com cuidado, pois pode implicar uma relação linear que pode não ser adequada para todas as variáveis categóricas.
 
2. > Codificação de Números Inteiros: Se a variável categórica tiver categorias que podem ser mapeadas diretamente para valores numéricos (por exemplo, mês do ano: janeiro = 1, fevereiro = 2, etc.), você pode usar essa codificação.

3. > Regressão Linear Generalizada (Generalized Linear Regression): Em alguns casos, é possível estender a regressão linear para acomodar variáveis categóricas sem codificação. Isso é feito por meio de técnicas de Regressão Linear Generalizada (GLM), como a Regressão Linear com Efeitos Fixos ou a Regressão Linear com Variáveis Dummies.

##### $\large{\color{RoyalBlue}{5.1.}}$ $\large{\color{CadetBlue}{\textbf{Codificação Dummy ✔️}}}$

In [ ]:
temp = pd.DataFrame(dados)

# Aplicando a codificação dummy na coluna "Sex" e "Embarked"
dummy = pd.get_dummies(temp, columns=["Sex","Embarked"])

# Exibindo o DataFrame original e o DataFrame após a codificação dummy
print("DataFrame original:")
print(temp)

print("\nDataFrame após codificação dummy:")
print(dummy)

###### $\large{\color{RoyalBlue}{5.1.1}}$ $\large{\color{CadetBlue}{\textbf{Resultado do Modelo de LR com Codificação Dummy ✔️}}}$

In [ ]:
#Definindo variáveis preditoras e variável target
y = dummy['Survived']
x = dummy.drop('Survived', axis=1)

#Separando os dados em treino e teste
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y,test_size = 0.3)

#Criando o modelo
modelo = LinearRegression()
modelo.fit(x_treino,y_treino)

#Calculando o coeficiente R2
resultado = modelo.score(x_teste,y_teste)
print(resultado)

##### $\large{\color{RoyalBlue}{5.2.}}$ $\large{\color{CadetBlue}{\textbf{Atribuição Arbitrária para Valores Categóricos ❌}}}$

In [ ]:
df1 = pd.DataFrame(dados)

# Mapeando os valores usando um dicionário
sex_mapping = {
    "female": 1,
    "male": 2
}

# Substituindo os valores da coluna "Sex" usando o método "map"
df1["Sex"] = df1["Sex"].map(sex_mapping)

# Resultado final
df1.head()

O mesmo processo se repetiu com os dados da origem do embarque, a coluna Embarked, gerando o dataframe df2.

In [ ]:
df2 = pd.DataFrame(df1)

# Mapeando os valores usando um dicionário
place_mapping = {
    "S": 1,
    "C": 2,
    "Q":3
}

# Substituindo os valores da coluna "Sex" usando o método "map"
df2["Embarked"] = df2["Embarked"].map(place_mapping)

# Resultado final
df2.head()

Com os dados tratados, segui com a aplicação do modelo de Regressão Linear.

###### $\large{\color{RoyalBlue}{5.2.}}$ $\large{\color{CadetBlue}{\textbf{Resultado do Modelo de LR com Atribuição Arbitrária ❌}}}$ 

In [ ]:
#Definindo variáveis preditoras e variável target
y = df2['Survived']
x = df2.drop('Survived', axis=1)

#Separando os dados em treino e teste
x_treino,x_teste,y_treino,y_teste = train_test_split(x,y,test_size = 0.3)

#Criando o modelo
modelo = LinearRegression()
modelo.fit(x_treino,y_treino)

#Calculando o coeficiente R2
resultado = modelo.score(x_teste,y_teste)
print(resultado)

## $\large{\color{RoyalBlue}{6.}}$ $\large{\color{CadetBlue}{\textbf{Conclusão 🏁}}}$


Embora as duas abordagens não tenham dado uma diferença muito grande, sendo de 5% pelo método correto comparado ao incorreto, é importante tomar sempre a estratégia correta para cada conjunto de dado,além disso, para todos os casos o modelo de Linear Regression deu um resultado bem baixo comparado a modelos mais adequados como o Random Forest, a razão disto é que o Linear Regression é um modelo de variáveis contínuas que melhora seus resultados a medida que a correlação entre os dados da variável dependente e independente é mais forte, principalmente quando esses eixos tratam de uma única dimensão, o que não foi o caso deste exercício. 

* Linear Regression: 39%
* Random Forest: 77% [exercício aqui](https://www.kaggle.com/code/wesleypdasilva/desafio-titanic-random-forest)